In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataclasses import dataclass
from lib.clue import Clue
from lib.utils import black_print
import math
import numpy as np
from ortools.sat.python import cp_model
import random
from typing import NewType

Suspect = NewType("Suspect", str)
Weapon = NewType("Weapon", str)
Room = NewType("Room", str)
Motive = NewType("Motive", str)
Time = NewType("Time", str)

num_players = random.randint(3, 6)
num_weapons = max(
    3,
    min(
        num_players + random.randint(-1, 5),
        len(Clue.weapons),
    ),
)
num_suspects = min(num_weapons + random.randint(0, num_weapons - 1), len(Clue.suspects))
num_rooms = min(num_suspects + random.randint(0, num_suspects - 2), len(Clue.rooms))
suspects = [Suspect(s) for s in random.sample(Clue.suspects, k=num_suspects)]
weapons = [Weapon(w) for w in random.sample(Clue.weapons, k=num_weapons)]
rooms = [Room(r) for r in random.sample(Clue.rooms, k=num_rooms)]
if random.random() < 0.1:
    motives = [
        Motive(m)
        for m in random.sample(
            Clue.motives,
            k=max(3, min(num_weapons + random.randint(-1, 3), len(Clue.motives))),
        )
    ]
frequency = random.choice([0.25, 0.5, 1.0])
start = 24.0 - frequency
end = 0.0
for _ in range(random.randint(1, num_weapons)):
    if random.randint(0, 1):
        end += frequency
    else:
        start -= frequency


def format_time(time: float) -> str:
    return f"{int(time):02d}:{int(60 * (time - int(time))):02d}"


times = [
    Time(t)
    for t in Clue.get_times(
        format_time(start), format_time(end), f"{int(frequency * 60)}min"
    )
]
board_columns = {
    3: 2,
    4: 2,
    5: random.choice([2, 3]),
    6: random.choice([2, 3]),
    7: 3,
    8: 3,
    9: 3,
    10: random.choice([3, 4]),
    11: random.choice([3, 4]),
    12: random.choice([3, 4]),
    13: 4,
    14: 4,
    15: 4,
    16: 4,
    17: 4,
}[len(rooms)]
board_rows = math.ceil(len(rooms) / board_columns)
room_coords = {
    room: (id % board_columns, board_rows - id // board_columns)
    for id, room in enumerate(rooms)
}
coord_rooms = {v: k for k, v in room_coords.items()}
len(rooms), board_rows, board_columns, room_coords, coord_rooms

(12,
 4,
 3,
 {'Study': (0, 4),
  'Conservatory': (1, 4),
  'Lounge': (2, 4),
  'Carriage House': (0, 3),
  'Gazebo': (1, 3),
  'Hall': (2, 3),
  'Drawing Room': (0, 2),
  'Trophy Room': (1, 2),
  'Cloak Room': (2, 2),
  'Dining Room': (0, 1),
  'Billiard Room': (1, 1),
  'Kitchen': (2, 1)},
 {(0, 4): 'Study',
  (1, 4): 'Conservatory',
  (2, 4): 'Lounge',
  (0, 3): 'Carriage House',
  (1, 3): 'Gazebo',
  (2, 3): 'Hall',
  (0, 2): 'Drawing Room',
  (1, 2): 'Trophy Room',
  (2, 2): 'Cloak Room',
  (0, 1): 'Dining Room',
  (1, 1): 'Billiard Room',
  (2, 1): 'Kitchen'})

In [3]:
def random_suspect_time_rooms() -> dict[Time, Room]:
    room = random.choice(rooms)
    time_rooms = {times[0]: room}
    for time in times[1:]:
        if random.random() < 0.5:
            coords = room_coords[room]
            room = random.choice(
                [
                    room
                    for room in (
                        coord_rooms.get((coords[0] + x, coords[1] + y))
                        for x, y in [(-1, 0), (1, 0), (0, -1), (0, 1)]
                    )
                    if room is not None
                ]
            )
        time_rooms[time] = room
    return time_rooms

In [4]:
suspect_time_rooms = {suspect: random_suspect_time_rooms() for suspect in suspects}
black_print(suspect_time_rooms)

{
    "Colonel Mustard": {
        "11:30 PM": "Conservatory",
        "12:00 AM": "Conservatory",
        "12:30 AM": "Lounge",
        "01:00 AM": "Hall",
    },
    "Miss Peach": {
        "11:30 PM": "Kitchen",
        "12:00 AM": "Cloak Room",
        "12:30 AM": "Cloak Room",
        "01:00 AM": "Cloak Room",
    },
    "Mrs. Peacock": {
        "11:30 PM": "Drawing Room",
        "12:00 AM": "Carriage House",
        "12:30 AM": "Drawing Room",
        "01:00 AM": "Carriage House",
    },
    "Madame Rose": {
        "11:30 PM": "Gazebo",
        "12:00 AM": "Conservatory",
        "12:30 AM": "Study",
        "01:00 AM": "Study",
    },
    "Sgt. Gray": {
        "11:30 PM": "Study",
        "12:00 AM": "Conservatory",
        "12:30 AM": "Conservatory",
        "01:00 AM": "Conservatory",
    },
    "Mrs. White": {
        "11:30 PM": "Gazebo",
        "12:00 AM": "Gazebo",
        "12:30 AM": "Hall",
        "01:00 AM": "Lounge",
    },
    "Monsieur Brunette": {
        "11:

In [8]:
time_room_suspects = {
    time: {room: set[Suspect]() for room in rooms} for time in times
}
for suspect, time_rooms in suspect_time_rooms.items():
    for time, room in time_rooms.items():
        time_room_suspects[time][room].add(suspect)
black_print(time_room_suspects)

{
    "11:30 PM": {
        "Study": {"Sgt. Gray"},
        "Conservatory": {"Colonel Mustard"},
        "Lounge": set(),
        "Carriage House": set(),
        "Gazebo": {"Madame Rose", "Mrs. White"},
        "Hall": set(),
        "Drawing Room": {"Mrs. Peacock"},
        "Trophy Room": {"Miss Scarlet"},
        "Cloak Room": set(),
        "Dining Room": set(),
        "Billiard Room": set(),
        "Kitchen": {"Miss Peach", "Monsieur Brunette"},
    },
    "12:00 AM": {
        "Study": set(),
        "Conservatory": {"Madame Rose", "Colonel Mustard", "Sgt. Gray"},
        "Lounge": set(),
        "Carriage House": {"Mrs. Peacock"},
        "Gazebo": {"Mrs. White"},
        "Hall": set(),
        "Drawing Room": set(),
        "Trophy Room": set(),
        "Cloak Room": {"Miss Peach", "Monsieur Brunette"},
        "Dining Room": set(),
        "Billiard Room": {"Miss Scarlet"},
        "Kitchen": set(),
    },
    "12:30 AM": {
        "Study": {"Madame Rose"},
        "Conserva

In [6]:
@dataclass
class WeaponMove:
    weapon: Weapon
    suspect: Suspect
    time: Time
    from_room: Room
    to_room: Room


# Storing these is not strictly necessary, but it's easier than deriving them after the fact
weapon_moves: list[WeaponMove] = []


def random_weapon_time_rooms(weapon: Weapon) -> dict[Time, Room]:
    room = random.choice(rooms)
    time_rooms = {times[0]: room}
    suspect: Suspect | None = None
    for time in times[1:]:
        if (
            suspect
            and (suspect_room := suspect_time_rooms[suspect][time])
            and suspect_room != room
        ):
            # Weapon moved by the suspect to a different room
            weapon_moves.append(
                WeaponMove(
                    weapon=weapon,
                    suspect=suspect,
                    time=time,
                    from_room=room,
                    to_room=suspect_room,
                )
            )
            time_rooms[time] = room = suspect_room
        else:
            # There is no suspect or the suspect hasn't moved
            time_rooms[time] = room
        if suspects := time_room_suspects[time][room]:
            suspect = random.choice(list(suspects))
    return time_rooms


weapon_time_rooms = {weapon: random_weapon_time_rooms(weapon) for weapon in weapons}
black_print(weapon_moves)
black_print(weapon_time_rooms)

[
    WeaponMove(
        weapon="Knife",
        suspect="Mrs. White",
        time="01:00 AM",
        from_room="Hall",
        to_room="Lounge",
    ),
    WeaponMove(
        weapon="Candlestick",
        suspect="Mrs. Peacock",
        time="01:00 AM",
        from_room="Drawing Room",
        to_room="Carriage House",
    ),
    WeaponMove(
        weapon="Revolver",
        suspect="Mrs. Peacock",
        time="01:00 AM",
        from_room="Drawing Room",
        to_room="Carriage House",
    ),
    WeaponMove(
        weapon="Horseshoe",
        suspect="Miss Scarlet",
        time="12:30 AM",
        from_room="Billiard Room",
        to_room="Dining Room",
    ),
    WeaponMove(
        weapon="Horseshoe",
        suspect="Miss Scarlet",
        time="01:00 AM",
        from_room="Dining Room",
        to_room="Drawing Room",
    ),
    WeaponMove(
        weapon="Lead Pipe",
        suspect="Mrs. Peacock",
        time="12:30 AM",
        from_room="Carriage House",
       

In [9]:
time_room_weapons = {
    time: {room: set[Weapon]() for room in rooms} for time in times
}
for weapon, time_rooms in weapon_time_rooms.items():
    for time, room in time_rooms.items():
        time_room_weapons[time][room].add(weapon)
black_print(time_room_weapons)

{
    "11:30 PM": {
        "Study": set(),
        "Conservatory": set(),
        "Lounge": set(),
        "Carriage House": {"Lead Pipe"},
        "Gazebo": set(),
        "Hall": {"Knife"},
        "Drawing Room": {"Revolver", "Candlestick"},
        "Trophy Room": {"Rope"},
        "Cloak Room": set(),
        "Dining Room": set(),
        "Billiard Room": {"Horseshoe"},
        "Kitchen": {"Wrench", "Poison"},
    },
    "12:00 AM": {
        "Study": set(),
        "Conservatory": set(),
        "Lounge": set(),
        "Carriage House": {"Lead Pipe"},
        "Gazebo": set(),
        "Hall": {"Knife"},
        "Drawing Room": {"Revolver", "Candlestick"},
        "Trophy Room": {"Rope"},
        "Cloak Room": set(),
        "Dining Room": set(),
        "Billiard Room": {"Horseshoe"},
        "Kitchen": {"Wrench", "Poison"},
    },
    "12:30 AM": {
        "Study": set(),
        "Conservatory": set(),
        "Lounge": set(),
        "Carriage House": set(),
        "Gazebo": s

In [24]:
while True:
    murderer = random.choice(suspects)
    murder_time = random.choice(times)
    murder_room = suspect_time_rooms[murderer][murder_time]
    if weapons := time_room_weapons[murder_time][murder_room]:
        murder_weapon = random.choice(list(weapons))
        break

black_print(
    {
        "murderer": murderer,
        "murder_weapon": murder_weapon,
        "murder_room": murder_room,
        "murder_time": murder_time,
    }
)

{
    "murderer": "Miss Scarlet",
    "murder_weapon": "Horseshoe",
    "murder_room": "Billiard Room",
    "murder_time": "12:00 AM",
}
